In [19]:
import numpy as np
import pandas as pd
from scipy.stats import norm

DATA_PATH = "data/spx_returns_weekly.xlsx"   # uploaded file
SHEET_NAME = "s&p500 rets"                        # correct sheet
ALPHA = 0.05
ROLL_WIN_WEEKS = 52                               # rolling vol window
HORIZONS_WEEKS = [1, 4]                           # VaR/CVaR horizons (weeks)
TICKERS_PREFERRED = ["AAPL", "META", "NVDA", "TSLA"]



In [ ]:
#1.1

In [ ]:
#1.2

In [ ]:
#1.3

In [25]:
# ---- load weekly returns ----
rets = pd.read_excel(DATA_PATH, sheet_name=SHEET_NAME, index_col=0, parse_dates=True)

# clean
rets = rets.apply(pd.to_numeric, errors="coerce").dropna(how="all").sort_index()
rets = rets.loc[~rets.index.duplicated(keep="first")]

# choose 4 tickers
available = [t for t in TICKERS_PREFERRED if t in rets.columns]
if len(available) < 4:
    available = list(rets.columns[:4])
tickers = available[:4]

# subset and make equally-weighted portfolio
rets_sel = rets[tickers].dropna(how="any")
w = np.repeat(1/len(tickers), len(tickers))
rp = rets_sel.dot(w)
rp.name = "EW"


In [22]:
# 2.0 HELPERS

def normal_var_cvar(sigma, alpha=0.05, mu=0.0, horizon_weeks=1):
    """
    Normal VaR/CVaR at tail alpha (return units), sqrt(h) scaling.
    VaR is alpha-quantile; CVaR is conditional mean below VaR.
    """
    z = norm.ppf(alpha)
    sig_h = sigma * np.sqrt(horizon_weeks)
    var = mu + sig_h * z
    cvar = mu - sig_h * norm.pdf(z) / alpha
    return var, cvar

def empirical_var_cvar(x, alpha=0.05):
    """ Historical (empirical) VaR & CVaR for a return series x. """
    x = pd.Series(x).dropna()
    var = x.quantile(alpha, interpolation="linear")
    cvar = x[x <= var].mean()
    return var, cvar

In [23]:
# 2.1 CONDITIONAL STATS (END-OF-SAMPLE)

# rolling weekly vol for EW portfolio
roll_std_ew = rp.rolling(window=ROLL_WIN_WEEKS, min_periods=ROLL_WIN_WEEKS).std(ddof=1)

# end-of-sample weekly & annualized vol
sigma_w_end = roll_std_ew.iloc[-1]
ann_factor_weeks = np.sqrt(52)
sigma_ann_end = sigma_w_end * ann_factor_weeks

# normal VaR/CVaR at end using rolling vol (1w & 4w)
rows = []
for h in HORIZONS_WEEKS:
    var_h, cvar_h = normal_var_cvar(sigma=sigma_w_end, alpha=ALPHA, mu=0.0, horizon_weeks=h)
    rows.append({
        "Horizon (weeks)": h,
        "Annualized Vol (as of end)": sigma_ann_end,
        f"Normal VaR {int(ALPHA*100)}%": var_h,
        f"Normal CVaR {int(ALPHA*100)}%": cvar_h
    })
cond_table = pd.DataFrame(rows).set_index("Horizon (weeks)")

# STATIC (full-sample) stats for comparison
sigma_w_full = rp.std(ddof=1)
sigma_ann_full = sigma_w_full * ann_factor_weeks
var_emp, cvar_emp = empirical_var_cvar(rp, alpha=ALPHA)

compare = pd.DataFrame({
    "Weekly Vol (full sample)": [sigma_w_full],
    "Annualized Vol (full sample)": [sigma_ann_full],
    f"Empirical VaR {int(ALPHA*100)}% (full sample)": [var_emp],
    f"Empirical CVaR {int(ALPHA*100)}% (full sample)": [cvar_emp]
}, index=["EW portfolio"])

print("\n--- 2.1: Conditional (end-of-sample) stats for EW portfolio ---")
display(cond_table)

print("\n--- Comparison to static (full-sample) stats from 1.2 ---")
display(compare)


--- 2.1: Conditional (end-of-sample) stats for EW portfolio ---


,Annualized Vol (as of end),Normal VaR 5%,Normal CVaR 5%
Horizon (weeks),,,
1,0.284408,-0.064873,-0.081354
4,0.284408,-0.129747,-0.162708



--- Comparison to static (full-sample) stats from 1.2 ---


,Weekly Vol (full sample),Annualized Vol (full sample),Empirical VaR 5% (full sample),Empirical CVaR 5% (full sample)
EW portfolio,0.043077,0.31063,-0.062054,-0.085026


In [24]:
# 2.2 BACKTEST: HIT RATES (1-week horizon)

alpha = ALPHA
z = norm.ppf(alpha)

# Expanding weekly vol through t
exp_std_ew = rp.expanding(min_periods=20).std(ddof=1)
var_exp_t = z * exp_std_ew                   # VaR_t using data through t
hits_exp = rp.shift(-1) < var_exp_t          # hit if r_{t+1} < VaR_t
hit_rate_exp = hits_exp.dropna().mean()

# Rolling weekly vol (52w window)
var_roll_t = z * roll_std_ew
hits_roll = rp.shift(-1) < var_roll_t
hit_rate_roll = hits_roll.dropna().mean()

bt = pd.DataFrame({
    "Hit Rate (expanding vol)": [hit_rate_exp],
    "Obs (expanding)": [hits_exp.dropna().shape[0]],
    "Hit Rate (rolling vol)": [hit_rate_roll],
    "Obs (rolling)": [hits_roll.dropna().shape[0]],
}, index=["EW portfolio"])

print("\n--- 2.2: VaR hit-test results (alpha = {:.0%}, horizon = 1 week) ---".format(alpha))
display(bt)


--- 2.2: VaR hit-test results (alpha = 5%, horizon = 1 week) ---


,Hit Rate (expanding vol),Obs (expanding),Hit Rate (rolling vol),Obs (rolling)
EW portfolio,0.052314,497,0.030181,497
